In [10]:
from __future__ import division

import numpy as np
import os
import tempfile

import keras
from keras import backend as K
from keras import layers
from keras.datasets import mnist

import tensorflow as tf
from tensorflow.contrib.data import Dataset
# import tensorflow.data.Dataset

In [11]:
def cnn_layers(inputs):
    x = layers.Conv2D(32, (3, 3),
                      activation='relu', padding='valid')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
#     predictions = layers.Dense(num_classes,
#                                activation='softmax',
#                                name='x_train_out')(x)

#1. the last layer has be modified for regression
    predictions = layers.Dense(1)(x)
    return predictions


In [12]:
batch_size = 128
buffer_size = 10000
steps_per_epoch = np.ceil(60000 / 128).astype('int')  # = 469
epochs = 5

#2 no classes, since we will do regression
# num_classes = 10


In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype(np.float32) / 255
x_train = np.expand_dims(x_train, -1)

# y_train = tf.one_hot(y_train, num_classes)

#2. the y_train has been converted to float32 for regression
y_train = y_train.astype(np.float32)
y_train = np.expand_dims(y_train, -1)


In [14]:
# Create the dataset and its associated one-shot iterator.
dataset = Dataset.from_tensor_slices((x_train, y_train))

dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size)

iterator = dataset.make_one_shot_iterator()


In [15]:
# Model creation using tensors from the get_next() graph node.
inputs, targets = iterator.get_next()

model_input = layers.Input(tensor=inputs)
model_output = cnn_layers(model_input)
train_model = keras.models.Model(inputs=model_input, outputs=model_output)


In [16]:
# train_model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),
#                     loss='categorical_crossentropy',
#                     metrics=['accuracy'],
#                     target_tensors=[targets])
# train_model.compile(loss='mean_squared_error', optimizer='adam')

#3. the model has been compliled with rmsprop optimizer with mse loss for regression
train_model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
train_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               819712    
__________

In [17]:
#4. the model fit now gets AttributeError: 'NoneType' object has no attribute 'shape'

train_model.fit(epochs=epochs, batch_size=batch_size, steps_per_epoch=steps_per_epoch)

AttributeError: 'NoneType' object has no attribute 'shape'